In [ ]:
import torch
from torch import nn
import random
from torch import nn
from torch.utils.data import Dataset,DataLoader
from sklearn.metrics import accuracy_score,confusion_matrix
import numpy as np
import sys
import torch.nn.functional as F
import copy
import pathlib
from pathlib import Path
import os


In [ ]:
import torch
import numpy as np
import random

# Function to set seed for all libraries
def set_seed(seed=0):
    # Set the seed for NumPy
    np.random.seed(seed)

    # Set the seed for random library
    random.seed(seed)

    # Set the seed for PyTorch
    torch.manual_seed(seed)

    # If you're using GPU, also set the seed for CUDA
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)  # for multi-GPU systems

    # To ensure that PyTorch's deterministic behavior is maintained
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

# Apply the seed at the beginning of your script
set_seed(0)

# Continue with your model training, dataset loading, etc.


In [ ]:
np.__version__

'1.26.4'

In [ ]:
device="cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [ ]:
Path.cwd()

PosixPath('/content')

In [ ]:
save_path=Path.cwd()/"result.txt"

if save_path.is_dir():
    os.remove(save_path)



In [ ]:
import scipy.signal as signal
import scipy.io as io

In [ ]:
import pathlib
from pathlib import Path

In [ ]:
device="cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [ ]:


class LightweightConv1d(nn.Module):
    """
    Args:
        input_size: # of channels of the input and output
        kernel_size: convolution channels
        padding: padding
        num_heads: number of heads used. The weight is of shape
            `(num_heads, 1, kernel_size)`
        weight_softmax: normalize the weight with softmax before the convolution
    Shape:
        Input: BxCxT, i.e. (batch_size, input_size, timesteps)
        Output: BxCxT, i.e. (batch_size, input_size, timesteps)
    Attributes:
        weight: the learnable weights of the module of shape
            `(num_heads, 1, kernel_size)`
        bias: the learnable bias of the module of shape `(input_size)`
    """
    def __init__(self, input_size, kernel_size=1, padding=0, num_heads=1, weight_softmax=False,
                 bias=False,dilation=1):
        super().__init__()
        self.input_size = input_size
        self.kernel_size = kernel_size
        self.num_heads = num_heads
        self.padding = padding
        self.weight_softmax = weight_softmax
        self.weight = nn.Parameter(torch.Tensor(num_heads, 1, kernel_size))
        self.dilation=dilation
        if bias:
            self.bias = nn.Parameter(torch.Tensor(input_size))
        else:
            self.bias = None

        self.init_parameters()

    def init_parameters(self):
        nn.init.xavier_uniform_(self.weight)
        if self.bias is not None:
            nn.init.constant_(self.bias, 0.0)

    def forward(self, input):
        B, C, T = input.size()
        H = self.num_heads

        weight = self.weight
        if self.weight_softmax:
            weight = F.softmax(weight, dim=-1)

        input = input.view(-1, H, T)
        output = F.conv1d(input, weight,dilation=self.dilation, padding=self.padding, groups=self.num_heads)
        output = output.view(B, C, -1)
        if self.bias is not None:
            output = output + self.bias.view(1, -1, 1)

        return output






class RawConvNet(nn.Module):
    def __init__(self, num_classes=4, num_samples=1000, num_channels=22, num_bands=9, embed_dim=32,
                 win_len=250, num_heads=4, weight_softmax=True, bias=False):
        super().__init__()
        self.win_len=win_len
        self.temporal_filter=nn.Conv2d(1,num_bands,kernel_size=(1,64),padding='same')
        self.spacial_block=nn.Sequential(
            nn.Conv2d(in_channels=num_bands,out_channels=embed_dim,kernel_size=(22,1)),
            nn.BatchNorm2d(embed_dim),
            nn.ELU())

        self.conv = LightweightConv1d(input_size=embed_dim, kernel_size= (num_samples//win_len), num_heads=num_heads,
                                     weight_softmax=weight_softmax, bias=bias,dilation=1
                                                             )
        self.classify = nn.Linear(embed_dim, num_classes)
    def forward(self,x):
        out=x.unsqueeze(dim=1)
        out=self.temporal_filter(out)
        out = self.spacial_block(out)
        out = out.reshape([*out.shape[0:2], -1, self.win_len])
      #  print(out.shape)
        out = torch.log(torch.clamp(out.var(dim=3),1e-6,1e6))
      #  out = torch.clamp(out.var(dim=3),1e-6,1e6)
        out = self.conv(out)
        out = out.view(out.size(0), -1)
        out = self.classify(out)

        return out

In [ ]:
#

In [ ]:
from torch.utils.data import Dataset

class eegDataset(Dataset):
    def __init__(self, data, label, transform=None):
        super().__init__()
        self.data = data
        self.labels = label
        self.transform = transform

    def __getitem__(self, index):
        if self.transform is None:
            data = self.data[index]
        else:
            data = self.transform(self.data[index])

        label = self.labels[index]

        return data, label

    def __len__(self):
        return len(self.data)

    def combineDataset(self, otherDataset):
        self.labels = np.hstack((self.labels, otherDataset.labels))
        self.data = np.concatenate((self.data, otherDataset.data), axis=0)

In [ ]:
def shuffle_data(data, label):
    print(data.shape)
    index = [i for i in range(len(data))]
    random.shuffle(index)
    shuffle_data = data[index]
    shuffle_label = label[index]
    return shuffle_data, shuffle_label

In [ ]:
from sklearn.model_selection import StratifiedKFold

def generateBalancedFolds(idx, label, kFold=10):
    folds = []
    skf = StratifiedKFold(n_splits=kFold)
    for train_index, test_index in skf.split(idx, label):
        folds.append([idx[i] for i in test_index])
    return folds

In [ ]:

    # Set the random seed for reproducibility
    def set_seed(seed):
        random.seed(seed)  # Python's built-in random module
        np.random.seed(seed)  # Numpy random seed
        torch.manual_seed(seed)  # Set seed for CPU
        if torch.cuda.is_available():
            torch.cuda.manual_seed_all(seed)  # Set seed for all GPUs

    # Set seed for reproducibility
    set_seed(0)











    lr = 0.001
    lrFactor = 0.6
    lrPatience = 20
    lrMin = 0.0001






    # Define network parameters
    num_classes = 4
    num_samples = 1000
    num_channels = 22
    num_bands = 9
    embed_dim = 64
    win_len = 20
    num_heads = 64
    weight_softmax = True
    bias = False

    mymodel = RawConvNet(num_classes=num_classes,
        num_samples=num_samples,
        num_channels=num_channels,
        num_bands=num_bands,
        embed_dim=embed_dim,
        win_len=win_len,
        num_heads=num_heads,
        weight_softmax=weight_softmax,
        bias=bias).to(device)

    loss_fn = nn.CrossEntropyLoss()

In [ ]:
x_check=torch.randn(19,22,1000).to(device)
mymodel(x_check).shape

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/conv.py:454: UserWarning: Using padding='same' with even kernel lengths and odd dilation may require a zero-padded copy of the input be created (Triggered internally at ../aten/src/ATen/native/Convolution.cpp:1031.)
  return F.conv2d(input, weight, bias, self.stride,


torch.Size([19, 4])

In [ ]:
def train_step(mymodelstuff):

    model,dataloader,loss_fn,optimizer,scheduler,device=mymodelstuff["model"],mymodelstuff['train_dataloader'],mymodelstuff['loss_fn'],mymodelstuff['optimizer'],mymodelstuff['scheduler'],mymodelstuff['device']

    model.train()

    train_loss=0
    train_accuracy=0

    for X,y in dataloader:
        X,y=X.to(device),y.to(device)

        y_pred=model(X)
        loss=loss_fn(y_pred,y)

        train_loss=loss+train_loss
        train_accuracy=train_accuracy+accuracy_score(y.to('cpu'),y_pred.argmax(1).to('cpu'))

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    train_loss=train_loss/len(dataloader)
    train_accuracy=train_accuracy/len(dataloader)
    return train_loss,train_accuracy

In [ ]:
torch.zeros(4,4)

tensor([[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]])

In [ ]:
def test_step(mymodelstuff):

    model,loss_fn,device,dataloader=mymodelstuff["model"],mymodelstuff['loss_fn'],mymodelstuff['device'],mymodelstuff['test_dataloader']

    model.eval()
    model.to(mymodelstuff["device"])

    confusion_matrix_result=torch.zeros(4,4)
    test_loss=0
    test_accuracy = 0
    confustion_matrx=torch.zeros(4,4)

    with torch.inference_mode():

        for X,y in dataloader:
            X,y=X.to(device),y.to(device)

            y_pred=model(X)

            loss=loss_fn(y_pred,y)

            test_loss=loss+test_loss
            test_accuracy=test_accuracy+accuracy_score(y.to('cpu'),y_pred.argmax(1).to('cpu'))
            confusion_matrix_result=confusion_matrix_result+confusion_matrix(y.to('cpu'),y_pred.argmax(1).to('cpu'))

    test_loss=test_loss/len(dataloader)
    test_accuracy=test_accuracy/len(dataloader)

    print(f"\n after training test accuracy is {test_accuracy}")
    return test_loss,test_accuracy,confusion_matrix

In [ ]:
def valid_step(mymodelstuff):

    model,loss_fn,device,dataloader=mymodelstuff["model"],mymodelstuff['loss_fn'],mymodelstuff['device'],mymodelstuff['valid_dataloader']
    model.eval()


    valid_loss=0
    valid_accuracy = 0

    with torch.inference_mode():

        for X,y in dataloader:
            X,y=X.to(device),y.to(device)

            y_pred=model(X)

            loss=loss_fn(y_pred,y)

            valid_loss=loss+valid_loss
            valid_accuracy=valid_accuracy+accuracy_score(y.to('cpu'),y_pred.argmax(1).to('cpu'))

    valid_loss=valid_loss/len(dataloader)
    valid_accuracy=valid_accuracy/len(dataloader)
    return valid_loss,valid_accuracy

model stopping with early stopping and stuff


In [ ]:
# function to combine train step and test step
# this function is train function

def train(mymodelstuff):
    results={"train_loss":[],"train_accuracy":[],"valid_loss":[],"valid_accuracy":[]}
    test_results={"test_loss":[],"test_accuracy":[],"test_confustion_matrix":[]}

    model,optimizer,scheduler=mymodelstuff['model'],mymodelstuff['optimizer'],mymodelstuff["scheduler"]
    num_epoch_after_early_stop=mymodelstuff['num_epoch_after_early_stop']
    max_epoch=mymodelstuff["max_epochs"]
    early_stop_reached=False
    continue_after_early_stop=mymodelstuff["continue_after_early_stop"]

    best_accuracy=0
    early_stop_count=mymodelstuff["early_stop_count"]
    stop_training=False
    count_epoch=0
    epoch=0

    while not stop_training:

        if not early_stop_reached:
            train_loss,train_accuracy=train_step(mymodelstuff=mymodelstuff)

            scheduler.step(train_loss)

            valid_loss,valid_accuracy=valid_step(mymodelstuff=mymodelstuff)



           ### print(f"\n epoch : {epoch} |Train_loss={train_loss}| Train Accuracy ={train_accuracy} | valid_loss : {valid_loss} | valid accuracy = {valid_accuracy}")
            #results["train_loss"].append(train_loss)
            #results["valid_loss"].append(valid_loss)

            #results["train_accuracy"].append(train_accuracy)
            #results["valid_accuracy"].append(valid_accuracy)


            if valid_accuracy>=best_accuracy:
                best_accuracy=valid_accuracy
                count_epoch=0
                best_parameters=copy.deepcopy(model.state_dict())
                best_optimizer=copy.deepcopy(optimizer.state_dict())
            else:
                count_epoch=count_epoch+1

            epoch=epoch+1


            if (count_epoch>=early_stop_count) or (epoch>=max_epoch):
                stop_training=True
                model.load_state_dict(best_parameters)
                optimizer.load_state_dict(best_optimizer)
                print(" \n early stop reached \n")


                if continue_after_early_stop:
                    print('\n continuing after early stopping\n')
                    stop_training=False
                    early_stop_reached=True
                    mymodelstuff["train_dataset"]=mymodelstuff["mixture_dataloader"]
                    epoch=0
                    scheduler._reset()
        else:

            train_loss,train_accuracy=train_step(mymodelstuff)
            scheduler.step(train_loss)
           # results['train_loss'].append(train_loss)
            epoch=epoch+1
           ### print(f"epoch is {epoch} train_loss is {train_loss} after early stop")
            if epoch>=num_epoch_after_early_stop:
                stop_training=True









    results["test_loss"],results["test_accuracy"],results["test_confustion_matrix"]=test_step(mymodelstuff)

    return results

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
path=Path('/content/drive/MyDrive/bcic')

Mounted at /content/drive


In [ ]:
X_train=np.load(path/'A01T_data.npy')
y_train=np.load(path/'A01T_label.npy')-1

X_test=np.load(path/"A01E_data.npy")
y_test=np.load(path/'A01E_label.npy').squeeze()-1


X_data,y_data=shuffle_data(data=X_train,label=y_train)

#k fold cross validation

k_folded_index=generateBalancedFolds(list(range(len(y_data))),y_data)

final_confusion_matrix=torch.tensor([])

(288, 22, 1000)


In [ ]:
accuracy_of_all_subjects=[]


for mn in np.arange(1,10,1):

    print(f'\n\n\n j values is {mn}')

    X_train=np.load(path/f'A0{mn}T_data.npy')
    y_train=np.load(path/f'A0{mn}T_label.npy')-1

    X_test=np.load(path/f"A0{mn}E_data.npy")
    y_test=np.load(path/f'A0{mn}E_label.npy').squeeze()-1


    X_data,y_data=shuffle_data(data=X_train,label=y_train)

    #k fold cross validation

    k_folded_index=generateBalancedFolds(list(range(len(y_data))),y_data)

    final_confusion_matrix=torch.tensor([])



    test_accuracy=[]

    for i,fold in enumerate(k_folded_index):
        print(f"\n this is {i+1} th fold training \n")
        k_folded_index_copy=copy.deepcopy(k_folded_index)
        test_index=fold
        if i+1<10:
            valid_index=[j for j in k_folded_index_copy[i+1]]
        else:
            valid_index=[j for j in k_folded_index_copy[0]]




        k_folded_index_copy.remove(valid_index)


        train_index=[]
        for f in k_folded_index_copy:
            for j in f:
                train_index.append(j)

        X_train=torch.tensor(np.array([X_data[j] for j in train_index])).to(torch.float32)
        y_train=torch.tensor(np.array([y_data[j] for j in train_index]))

        X_test=torch.tensor(X_test).to(torch.float32)
        y_test=torch.tensor(y_test)

        X_valid=torch.tensor(np.array([X_data[j] for j in valid_index])).to(torch.float32)
        y_valid=torch.tensor(np.array([y_data[j] for j in valid_index]))

        X_mixture=torch.concat((X_train,X_valid),dim=0)
        y_mixture=torch.concat((y_train,y_valid),dim=0)

        train_data_custom=eegDataset(X_train,y_train)
        test_data_custom=eegDataset(X_test,y_test)
        valid_data_custom=eegDataset(X_valid,y_valid)
        mixture_custom=eegDataset(X_mixture,y_mixture)
        #turn into dataloader


        from torch.utils.data import DataLoader
        BATCH_SIZE=16
        train_dataloader=DataLoader(dataset=train_data_custom,batch_size=BATCH_SIZE,shuffle=True)
        test_dataloader=DataLoader(dataset=test_data_custom,batch_size=BATCH_SIZE)
        valid_dataloader=DataLoader(dataset=valid_data_custom,batch_size=BATCH_SIZE)
        mixture_dataloader=DataLoader(dataset=mixture_custom,batch_size=BATCH_SIZE)
        mymodel1=copy.deepcopy(mymodel)

        optimizer = torch.optim.Adam(mymodel1.parameters(), lr=lr)
        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=lrFactor,
                                                        patience=lrPatience, min_lr=lrMin)
        mymodelstuff={
            "model":mymodel1,
            "train_dataloader":train_dataloader,
            "valid_dataloader":valid_dataloader,
            "test_dataloader":test_dataloader,
            "mixture_dataloader":mixture_dataloader,
            "optimizer":optimizer,
            "scheduler":scheduler,
            "loss_fn":loss_fn,
            "max_epochs":500,
            "num_epoch_after_early_stop":200,
            "continue_after_early_stop":True,
            "early_stop_count":50,
            "max_epoch_after_early_stop":200,
            "device":device,


        }

        #train model
        test_results=train(mymodelstuff=mymodelstuff)

        test_accuracy.append(test_results['test_accuracy'])

    accuracy_of_all_subjects.append(np.mean(test_accuracy))
    print(f'accuracy_of_all_subject is {accuracy_of_all_subjects}')

        #final_confusion_matrix=torch.concat((final_confusion_matrix,test_results["test_confusion_matrix"].unsqueeze(dim=0)),dim=0)
    #     with open(save_path,"a") as f:
    #         f.write(f"\n{i+1}th fold training happening\n")
    #         f.write(test_results)

    #cm=torch.mean(final_confusion_matrix,dim=0)
    # with open(save_path,"a") as f:
    #     f.write("\n this is final  result \n")
    #     f.write(cm)






 j values is 1
(288, 22, 1000)

 this is 1 th fold training 

 
 early stop reached 


 continuing after early stopping


 after training test accuracy is 0.8194444444444444

 this is 2 th fold training 



<ipython-input-25-c0710340f2f7>:50: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_test=torch.tensor(X_test).to(torch.float32)
<ipython-input-25-c0710340f2f7>:51: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_test=torch.tensor(y_test)


 
 early stop reached 


 continuing after early stopping


 after training test accuracy is 0.8333333333333334

 this is 3 th fold training 



<ipython-input-25-c0710340f2f7>:50: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_test=torch.tensor(X_test).to(torch.float32)
<ipython-input-25-c0710340f2f7>:51: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_test=torch.tensor(y_test)


 
 early stop reached 


 continuing after early stopping


 after training test accuracy is 0.8506944444444444

 this is 4 th fold training 



...

<ipython-input-25-c0710340f2f7>:50: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_test=torch.tensor(X_test).to(torch.float32)
<ipython-input-25-c0710340f2f7>:51: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_test=torch.tensor(y_test)


 
 early stop reached 


 continuing after early stopping


 after training test accuracy is 0.8506944444444444

 this is 6 th fold training 



<ipython-input-25-c0710340f2f7>:50: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_test=torch.tensor(X_test).to(torch.float32)
<ipython-input-25-c0710340f2f7>:51: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_test=torch.tensor(y_test)


 
 early stop reached 


 continuing after early stopping


 after training test accuracy is 0.8645833333333334

 this is 7 th fold training 



<ipython-input-25-c0710340f2f7>:50: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_test=torch.tensor(X_test).to(torch.float32)
<ipython-input-25-c0710340f2f7>:51: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_test=torch.tensor(y_test)


In [ ]:
np.mean(accuracy_of_all_subjects)

0.7809027777777778

In [ ]:
test_accuracy
np.mean(test_accuracy)


0.8451388888888888

In [ ]:
np.mean(test_accuracy)

nan

In [ ]:
accuracy_of_all_subjects

[0.8055555555555556,
 0.8229166666666667,
 0.8310185185185186,
 0.8324652777777779,
 0.8284722222222223,
 0.8258101851851852,
 0.8253968253968255,
 0.82421875,
 0.8248456790123456,
 0.8253472222222221,
 0.4583333333333333,
 0.4930555555555556,
 0.4953703703703704,
 0.5130208333333334,
 0.5263888888888889,
 0.525462962962963,
 0.5257936507936508,
 0.5269097222222223,
 0.5258487654320988,
 0.5211805555555556,
 0.9305555555555556,
 0.9236111111111112,
 0.920138888888889,
 0.9218750000000001,
 0.925,
 0.9236111111111112,
 0.9241071428571429,
 0.9240451388888888,
 0.9243827160493827,
 0.9246527777777779,
 0.625,
 0.6684027777777778,
 0.6956018518518517,
 0.7074652777777777,
 0.7145833333333332,
 0.7152777777777777,
 0.7227182539682538,
 0.7183159722222223,
 0.7183641975308643,
 0.7156250000000001,
 0.7083333333333334,
 0.7065972222222223,
 0.7210648148148149,
 0.7291666666666667,
 0.7305555555555556,
 0.7390046296296298,
 0.7435515873015873,
 0.7421875,
 0.7407407407407408,
 0.7385416666666

In [ ]:
a=torch.ones(2,2)

In [ ]:
torch.mean(a,dim=0)

In [ ]:
test_results["test_confustion_matrix"]

In [ ]:
check_matrix=torch.tensor([])

In [ ]:
check_matrix=torch.cat((check_matrix,torch.randn(4,4).unsqueeze(0)),dim=0)

In [ ]:
check_matrix

In [ ]:
a_layer=nn.Conv2d(1,9,kernel_size=(1,125),padding='same')


In [ ]:
x_check=torch.randn(288,1,22,1000)

In [ ]:
a_layer(x_check).shape
